In [22]:
import unittest

In [23]:
import sys
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/")
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/src")

In [24]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gym.spaces as gym_spaces
import gymnasium as gym  # overwrite OpenAI gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic_dreamer import ActorCriticDreamer
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
!python -V

Python 3.10.4


In [27]:
class TestAddition(unittest.TestCase):
    
    def addition(self,x,y):
        return x+y
    
    def test_addition(self):
        self.assertEqual(self.addition(1,2), 3)

In [19]:
# if __name__ == "__main__":
#     unittest.main()

hi


In [28]:
src.actor_critic_discrete.DiscreteActorCritic

src.actor_critic_discrete.DiscreteActorCritic

In [30]:
agent = src.actor_critic_discrete.DiscreteActorCritic()

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [31]:
agent.critic

MLP(
  (layers): Sequential(
    (layer_0): Sequential(
      (0): Linear(in_features=1536, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_2): Sequential(
      (0): Linear(in_features=256, out_features=255, bias=True)
      (2): Softmax(dim=-1)
    )
  )
)

In [45]:
sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])

In [47]:
value_pred, critic_dist = agent.apply_critic(sample_input)

In [50]:
len(value_pred.shape)

0

In [57]:
critic_dist.shape

torch.Size([255])

In [59]:
config["num_buckets"]

255

In [63]:
torch.Size([config["num_buckets"]])

torch.Size([255])

In [51]:
class TestDiscreteActorCritic(unittest.TestCase):
    
    agent = src.actor_critic_discrete.DiscreteActorCritic()
    
    # test if the critic softmax sums to 1
    # test the critic output shape 
    # => for a single instance
    # => for a batch
    def test_critic(self):
        
        sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])
        value_pred, critic_dist = agent.apply_critic(sample_input)
        
        # value_pred should be a scalar without shape
        self.assertEqual(len(value_pred.shape), 0)

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [65]:
sample_batch = torch.randn(32, config["H"] + config["Z"]).to(config["device"])
sample_batch.shape

torch.Size([32, 1536])

In [67]:
value_pred, critic_dist = agent.apply_critic(sample_batch)

In [72]:
torch.sum(critic_dist, dim=-1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', grad_fn=<SumBackward1>)